In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
from tqdm import tqdm
import string
import collections
import numpy as np

from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [2]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

suffix_df = "/raw/main/results/output/output_df.csv"
suffix_squadid_df = "/resolve/main/results/output/output_df.csv"

In [3]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Context', 'Question', 'Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

In [4]:
def find_sample(url, type):
    
    df = take_dataframe(url)
    
    rows = []

    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(df['Rec. Pred Answer'][i])[0] # Kandidat pertama
        nli_validated_answer = df['Prediction Answer'][i]
        
        if type == "ra":
        
            ra = (gold_answer == qa_generated_answer) and (qa_generated_answer == nli_validated_answer)

            if ra:
                rows.append(df.iloc[i])
                
        elif type == "rr":
        
            rr = (gold_answer == qa_generated_answer) and (qa_generated_answer != nli_validated_answer)

            if rr:
                rows.append(df.iloc[i])
        
        elif type == "wa":
        
            wa = (gold_answer != qa_generated_answer) and (qa_generated_answer == nli_validated_answer)

            if wa:
                rows.append(df.iloc[i])
                
        elif type == "wr":
        
            wr = (gold_answer != qa_generated_answer) and (qa_generated_answer != nli_validated_answer)

            if wr:
                rows.append(df.iloc[i])

    df = pd.DataFrame(rows).reset_index(drop=True)
    df = df[["Prediction Answer", "Gold Answer"]]

    return df

# idkmrc

In [5]:
data = "idkmrc"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
msi = "1"
var = "2"
th = "0.75"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_df}"

In [6]:
# Sama semua

df = pd.DataFrame(find_sample(url, type="ra")).head(20)
df

,Prediction Answer,Gold Answer
0,1959,1959
1,,
2,,
3,,
4,,
5,,
6,,
7,Januari 414,Januari 414
8,,
9,,


In [7]:
# Prediction kosong

df = pd.DataFrame(find_sample(url, type="rr")).head(20)
df

,Prediction Answer,Gold Answer
0,,20 tahun


In [8]:
# Beragam jenis, super/sub/intersection, ada juga string kosong

df = pd.DataFrame(find_sample(url, type="wa")).head(20)
df

,Prediction Answer,Gold Answer
0,komponen dari sebuah nama pribadi yang berdasa...,Komomene sebuah nama yang berdasarkan pada nam...
1,Gubernur DKI Jakarta,Gubernur DKI Jakarta yang menjabat sejak 19 No...
2,Ir. Basuki Tjahaja Purnama,Basuki Tjahaja Purnama
3,"Hispania, Ataulf dengan tidak hati-hati meneri...",istana Barcelona
4,Lontong kupang atau kupang lontong adalah nama...,
5,"1634 masehi, dan selesai pada tahun 1641","han dimulai pada tahun 1634 masehi, dan selesa..."
6,"provinsi Lecce (Apulia, Italia","Apulia, Italia"
7,"mengatur air, menurunkan hujan/petir, membagik...",
8,istilah yang dipakai baik secara populer maupu...,istilah yang dipakai baik secara populer maupu...
9,,Yesus Kristus


In [9]:
# Prediction selalu kosong

df = pd.DataFrame(find_sample(url, type="wr")).head(20)
df

,Prediction Answer,Gold Answer
0,,
1,,1889
2,,1920-an
3,,"1,120 tonnes (1,100 long tons) ketika berada d..."
4,,2009
5,,


# tydiqaid

In [10]:
data = "tydiqaid"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
msi = "2"
var = "3"
th = "0.25"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_df}"

In [11]:
# Sama semua

df = pd.DataFrame(find_sample(url, type="ra")).head(20)
df

,Prediction Answer,Gold Answer
0,Yoo Kerl,Yoo Kerl
1,24 Oktober 1945,24 Oktober 1945
2,"Sheffield, South Yorkshire","Sheffield, South Yorkshire"
3,"Joseph Patrick ""Joe"" Kennedy, Sr. (1888–1969) ...","Joseph Patrick ""Joe"" Kennedy, Sr. (1888–1969) ..."
4,Felis margarita,Felis margarita
5,"Nasi glepungan, Sirup pokak, keripik kentang","Nasi glepungan, Sirup pokak, keripik kentang"
6,Charles Babbage,Charles Babbage
7,Game Freak dan Creatures Inc,Game Freak dan Creatures Inc
8,Adolf von Thadden,Adolf von Thadden
9,Kyoko Mizuki,Kyoko Mizuki


In [12]:
# Salah tahun

df = pd.DataFrame(find_sample(url, type="rr")).head(20)
df

,Prediction Answer,Gold Answer
0,1974,1972


In [13]:
# Beragam jenis, super/sub/intersection, ada juga string kosong, 
# ada juga yang bener-bener salah semantik/format

df = pd.DataFrame(find_sample(url, type="wa")).head(20)
df

,Prediction Answer,Gold Answer
0,"TMP Cikutra, Bandung",28 Agustus 1950
1,"Puri Dragsholm, 75 kilometer Kopenhagen",Puri Dragsholm
2,14.300 km2.,14.300 km²
3,"suci, teristimewa oleh pemeluk agama Hindu",Hindu
4,Yosua bin Nun,"Yosua bin Nun, abdi Musa"
5,"Keluyuran di Pusat Mode New York”, yang merupa...",Keluyuran di Pusat Mode New York
6,menjaga standar mutu artistik yang tinggi dan ...,untuk menjaga standar mutu artistik yang tingg...
7,Sanskerta,bahasa Sanskerta
8,salah satu nama zaman pemerintahan kaisar Jepa...,nama zaman pemerintahan kaisar Jepang sewaktu ...
9,"hal-hal yang berkaitan dengan budi, dan akal m...","sebagai hal-hal yang berkaitan dengan budi, da..."


In [14]:
# Beragam jenis, super/sub/intersection
# dan ada juga yang bener-bener salah semantik/format

df = pd.DataFrame(find_sample(url, type="wr")).head(20)
df

,Prediction Answer,Gold Answer
0,<s> Siapa yang mengetuai Operation Just Cause?...,Amerika Serikat
1,padang rumput stepa Asia Tengah wilayah Mongol...,2008
2,<s> Apa isi di dalam lubang hitam?</s></s> Lub...,gravitasi paling kuat
3,empat puluh hari sejak Minggu Paskah (puncak d...,empat puluh hari sejak Minggu Paskah (puncak d...
4,luar negeri. Paspor ini hanya diberikan kepada...,Direktorat Jenderal Imigrasi
5,cabang biologi yang mempelajari pewarisan sifa...,cabang biologi yang mempelajari pewarisan sifa...
6,Teritori Seberang Laut Gibraltar. Baterai ini ...,ujung utara Upper Rock Nature Reserve
7,"bahasa percakapan Utara"" atau 北方方言 Hanyu Pinyi...",menerjemahkan beberapa istilah Tionghoa yang b...
8,Internet untuk Segala-(nya) (English: Internet...,IoT
9,Bahasa Tionghoa (dituturkan dalam bentuk stand...,Bahasa Tionghoa


# squadid

In [15]:
data = "squadid"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
msi = "2"
var = "3"
th = "0.5"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_squadid_df}"

In [16]:
# Sama semua

df = pd.DataFrame(find_sample(url, type="ra")).head(20)
df

,Prediction Answer,Gold Answer
0,Perancis,Perancis
1,"Denmark, Islandia dan Norwegia","Denmark, Islandia dan Norwegia"
2,Rollo,Rollo
3,abad ke-10,abad ke-10
4,William Sang Penakluk,William Sang Penakluk
5,"politik, budaya, dan militer","politik, budaya, dan militer"
6,Normandia,Normandia
7,Normandia,Normandia
8,abad ke-9,abad ke-9
9,abad ke-9,abad ke-9


In [17]:
# Beragam jenis, super/sub/intersection, ada juga string kosong, 
# ada juga yang bener-bener salah semantik/format

df = pd.DataFrame(find_sample(url, type="rr")).head(20)
df

,Prediction Answer,Gold Answer
0,solusi,solusinya
1,1900,1999
2,1806,1700
3,1562,1564
4,1643,1685
5,pertanian,pabrik
6,,3600 putaran per menit
7,Oksigen,oksigen
8,<s>,polutan
9,,"energi terbarukan, tenaga nuklir dan bahan bak..."


In [18]:
# Beragam jenis, super/sub/intersection, ada juga string kosong, 
# ada juga yang bener-bener salah semantik/format

df = pd.DataFrame(find_sample(url, type="wa")).head(20)
df

,Prediction Answer,Gold Answer
0,abad ke-10 dan ke-11,10 dan ke
1,Bangsa Normandia,Normandia
2,Normandia,Normanni
3,,Rollo
4,paruh pertama abad ke-10,abad ke-10
5,Richard I dari Normandia,Richard I
6,Kristen,Katolik
7,Richard I dari Normandia,Richard I
8,Kerajaan Sisilia,Antiokhia
9,"Norseman, Viking",Viking


In [19]:
# Beragam jenis, super/sub/intersection, ada juga string kosong, 
# ada juga yang bener-bener salah semantik/format

df = pd.DataFrame(find_sample(url, type="wr")).head(20)
df

,Prediction Answer,Gold Answer
0,Masalah komputasi,Masalah komputasi
1,kompleksitas,Langkah-langkah kompleksitas
2,<s>,Langkah-langkah kompleksitas
3,Variabel apa yang mudah dibangun dalam kerangk...,definisi rumit
4,<s>,NP-complete
5,terbalik,terbalik
6,terbalik,terbalik
7,algoritma waktu-eksponensial mungkin tidak dap...,algoritma waktu-eksponensial
8,yayasan,foundations were laid out
9,menjaga diskusi tetap abstrak agar tidak terga...,encoding
